In [ ]:
import geopandas as gpd
from ipyfilechooser import FileChooser
from pathlib import Path
import rasterio
import re
import requests
from shapely.geometry import Polygon, box 
from tqdm.auto import tqdm 

### Select the HLS data output from Notebook 0, which has been pre-processed and cropped to a chip

In [ ]:
print("Select the output preprocessed chip from Notebook 0")
fc = FileChooser(Path.cwd())
display(fc)

In [ ]:
chip_path = Path(fc.selected)
chip_path

### Grab the chip ID from the cropped image filename

In [ ]:
regex = "(?<=/cropped_to_chip_)\d+"
chip_id = int(re.search(regex, str(chip_path)).group(0))
chip_id

### Create a Polygon from the bounds of the chip 

In [ ]:
chips = gpd.read_file('ResampleblocksUpd.geojson')
geometry = chips.loc[chips['random_id'][chip_id], ['geometry']][0]

### Create an output directory for ESA WorldCover Data

In [ ]:
output_dir = Path.cwd()/"ESA_WorldCover_10m_2020_v1_data"
output_dir.mkdir(exist_ok=True)

### Download all ESA WorldCover tiles intersecting the chip

In [ ]:
s3_url_prefix = "https://esa-worldcover.s3.eu-central-1.amazonaws.com"

# load worldcover grid
url = f'{s3_url_prefix}/v100/2020/esa_worldcover_2020_grid.geojson'
grid = gpd.read_file(url)

# get grid tiles intersecting AOI
tiles = grid[grid.intersects(geometry)]

for tile in tqdm(tiles.ll_tile):
    url = f"{s3_url_prefix}/v100/2020/map/ESA_WorldCover_10m_2020_v100_{tile}_Map.tif\n"
    r = requests.get(url, allow_redirects=True)
    out_fn = output_dir/f"ESA_WorldCover_10m_2020_v100_{tile}_Map.tif"
    with open(out_fn, 'wb') as f:
        f.write(r.content)